In [1]:
import pandas as pd
import numpy as np

## 数据初探

### 导入数据

In [39]:
df = pd.read_csv("详情表.csv")
df.head()

,first_category,second_category,third_category,brand,price,good_percent,user_name,plus,star,content,...,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,￥ 158.00,97%,****G,False,comment-star star5,很好看的连衣裙，质量非常的好，穿起来很暖和。衣服尺码刚好合适。穿起来还挺显瘦的，朋友都说好看。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,￥ 158.00,97%,****n,False,comment-star star5,尺码很标准，上身效果很不错，客服态度很热情，价格很实惠。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,￥ 158.00,97%,****g,False,comment-star star5,质量真的是很不错，很满意的，包装很用心，客服态度特别的好，值得推荐。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,￥ 158.00,97%,****C,False,comment-star star5,衣服收到啦，套装简直是懒人福音，不用考虑搭配啦，质量很好，穿上特别显瘦，气质也很好，通勤日常...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,￥ 158.00,97%,****E,False,comment-star star5,面料很舒适，尺码大小很标准，做工很精细，客服态度也是特别的好。,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 对数据集有个基本印象

In [3]:
df.shape

(29817, 25)

#### 关键变量

In [4]:
# 查看我们的关键变量是否有缺失
df['price'].isnull().sum()

0

In [5]:
df['comment_time'].isnull().sum()

2176

## 数据清洗

### 缺失值处理

**删除空的列**

In [6]:
df.dropna(how='all', axis='columns',inplace=True)
df.shape

(29817, 13)

**有没有一种方式，能输入一个dataframe,然后直接输出一个表格，包含变量的缺失值和缺失率呢？**

In [7]:
# Define a function to visulize the features with missing values, and % of total values, & datatype
def missing_values_table(df):
     # Total missing values
    mis_val = df.isnull().sum()
    # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_type = df.dtypes
    # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent, mis_val_type], axis=1)
        
     # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values', 2: 'type'})
        
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[ mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        
    # Print some summary information
    print ("你所选择的数据有 " + str(df.shape[1]) + " 列.\n" "其中有 " + str(mis_val_table_ren_columns.shape[0]) + " 列的变量是有缺失值的.")
        
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [8]:
missing_values_table(df)

你所选择的数据有 13 列.
其中有 2 列的变量是有缺失值的.


,Missing Values,% of Total Values,type
size,2597,8.7,object
comment_time,2176,7.3,object


**咱们就以50%为界限，如果一个变量有超过50%的缺失的话，我们就将其抛弃**

In [9]:
missing_frac = df.isnull().mean()
drop_list = sorted(missing_frac[missing_frac > 0.50].index)

In [10]:
len(drop_list)

0

In [11]:
# 定义一个删除变量的函数，只要把要删除的变量列表放进去就行，方便之后使用
def drop_cols(cols):
    df.drop(labels=cols, axis=1, inplace=True)

In [12]:
drop_cols(drop_list)

In [13]:
missing_values_table(df)

你所选择的数据有 13 列.
其中有 2 列的变量是有缺失值的.


,Missing Values,% of Total Values,type
size,2597,8.7,object
comment_time,2176,7.3,object


In [14]:
# 删除行，具有缺失值的行。目前没有缺失行
def drop_null_row(cols):
    df.dropna(subset=cols, axis=0, inplace=True)

### 空值处理

**替换size中的空值为无**

In [15]:
df['size'].replace("",np.nan)
df['size'].fillna('无',inplace=True)
missing_values_table(df)

你所选择的数据有 13 列.
其中有 1 列的变量是有缺失值的.


,Missing Values,% of Total Values,type
comment_time,2176,7.3,object


**由于comment_time是我们一项重要的数据，所以空缺部分统一格式化，以避免在后续分析时对结果产生干扰**

In [16]:
df["comment_time"].isnull()
df.dropna(how='any',inplace=True)
df.shape

(27641, 13)

### 独特率

**这里我们不能再使用一个比例来剔除独特率高或者低的变量了，因为对于不同的变量，独特率高或者低都有可能是正常的**

In [17]:
df.nunique()

first_category         9
second_category       18
third_category        56
brand                331
price                279
good_percent          20
user_name           7189
plus                   2
star                   5
content            24567
style               1160
size                 653
comment_time       24122
dtype: int64

### 剔除重复值

**删除了完全重复的行，剩下的都是有效数据**

In [18]:
df = df.drop_duplicates()
df.shape

(26741, 13)

### 格式化

In [19]:
### 调整百分数

In [20]:
df['good_percent'] = df['good_percent'].str.strip('%').astype(float)/100
df['good_percent'] = df['good_percent'].apply(lambda x: format(x, '.2%'))
df['good_percent']

0        97.00%
1        97.00%
2        97.00%
3        97.00%
4        97.00%
          ...  
29812    98.00%
29813    98.00%
29814    98.00%
29815    98.00%
29816    98.00%
Name: good_percent, Length: 26741, dtype: object

In [21]:
df["star"] = df['star'].str.replace("comment-star star","")
df["star"] = pd.to_numeric(df["star"])
df['star']

0        5
1        5
2        5
3        5
4        5
        ..
29812    5
29813    5
29814    5
29815    5
29816    5
Name: star, Length: 26741, dtype: int64

In [22]:
df["price"] = df["price"].str.replace('￥ ', '')
df["content"] = df["content"].str.replace('\n', '')
# 转换数据类型为datetime
df['comment_time'] = pd.to_datetime(df['comment_time'])
df['comment_time'] = df['comment_time'].dt.date
df['comment_time'] = pd.to_datetime(df['comment_time'])
df['comment_time']

0       2022-10-24
1       2022-10-24
2       2022-10-23
3       2022-10-23
4       2022-10-22
           ...    
29812   2021-07-25
29813   2021-07-10
29814   2021-07-09
29815   2021-06-21
29816   2021-06-20
Name: comment_time, Length: 26741, dtype: datetime64[ns]

In [23]:
df

,first_category,second_category,third_category,brand,price,good_percent,user_name,plus,star,content,style,size,comment_time
0,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****G,False,5,很好看的连衣裙，质量非常的好，穿起来很暖和。衣服尺码刚好合适。穿起来还挺显瘦的，朋友都说好看。,两件套套装,M （建议100-110斤）,2022-10-24
1,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****n,False,5,尺码很标准，上身效果很不错，客服态度很热情，价格很实惠。,两件套套装,M （建议100-110斤）,2022-10-24
2,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****g,False,5,质量真的是很不错，很满意的，包装很用心，客服态度特别的好，值得推荐。,两件套套装,M （建议100-110斤）,2022-10-23
3,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****C,False,5,衣服收到啦，套装简直是懒人福音，不用考虑搭配啦，质量很好，穿上特别显瘦，气质也很好，通勤日常...,两件套套装,M （建议100-110斤）,2022-10-23
4,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****E,False,5,面料很舒适，尺码大小很标准，做工很精细，客服态度也是特别的好。,两件套套装,M （建议100-110斤）,2022-10-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29812,母婴,童鞋,儿童运动鞋,特步（XTEP）,139.90,98.00%,x***n,False,5,还不错，给弟弟买的，性价比高,黑白,38码,2021-07-25
29813,母婴,童鞋,儿童运动鞋,特步（XTEP）,139.90,98.00%,s***m,True,5,鞋子挺好的，小孩很喜欢。,黑白,38码,2021-07-10
29814,母婴,童鞋,儿童运动鞋,特步（XTEP）,139.90,98.00%,****Q,False,5,质量很好，舒服，不错的选择，大爱……,黑白,38码,2021-07-09
29815,母婴,童鞋,儿童运动鞋,特步（XTEP）,139.90,98.00%,****o,False,5,帮小儿子卖的，特步头一回购卖，质量如何，用了才知道。,黑白,38码,2021-06-21


拆分颜色

In [24]:
# df = pd.read_csv('1.csv',encoding="utf_8_sig")
styly_list = df['style'].str.split('色',n=1,expand=True)
df['color'] = styly_list[0]
df.head()

,first_category,second_category,third_category,brand,price,good_percent,user_name,plus,star,content,style,size,comment_time,color
0,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****G,False,5,很好看的连衣裙，质量非常的好，穿起来很暖和。衣服尺码刚好合适。穿起来还挺显瘦的，朋友都说好看。,两件套套装,M （建议100-110斤）,2022-10-24,两件套套装
1,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****n,False,5,尺码很标准，上身效果很不错，客服态度很热情，价格很实惠。,两件套套装,M （建议100-110斤）,2022-10-24,两件套套装
2,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****g,False,5,质量真的是很不错，很满意的，包装很用心，客服态度特别的好，值得推荐。,两件套套装,M （建议100-110斤）,2022-10-23,两件套套装
3,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****C,False,5,衣服收到啦，套装简直是懒人福音，不用考虑搭配啦，质量很好，穿上特别显瘦，气质也很好，通勤日常...,两件套套装,M （建议100-110斤）,2022-10-23,两件套套装
4,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****E,False,5,面料很舒适，尺码大小很标准，做工很精细，客服态度也是特别的好。,两件套套装,M （建议100-110斤）,2022-10-22,两件套套装


文字切词

In [25]:
import jieba
# df = pd.read_csv('1.csv',encoding="utf_8_sig")
# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('NLP停用词.txt')  # 这里加载停用词的路径
    outstr = ''
    for word in sentence_seged:
        if word not in stopwords:
            if word != '\t':
                outstr += word
                outstr += " "
    return outstr


# r = seg_sentence(df.iloc[0][10])
# print(r)
# s = seg_sentence(df.loc[0,'size'])
# print(s)
for index, row in df.iterrows():
    df.loc[index,'content'] = seg_sentence(row['content'])
    df.loc[index,'style'] = seg_sentence(row['style'])
    df.loc[index,'size'] = seg_sentence(row['size'])
df.head()

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\13169\AppData\Local\Temp\jieba.cache
Loading model cost 0.605 seconds.
Prefix dict has been built successfully.


,first_category,second_category,third_category,brand,price,good_percent,user_name,plus,star,content,style,size,comment_time,color
0,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****G,False,5,好看 连衣裙 质量 穿 暖和 衣服 尺码 合适 穿 挺 显瘦 朋友 说 好看,两件套 套装,M 建议 100 110 斤,2022-10-24,两件套套装
1,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****n,False,5,尺码 标准 上身 效果 不错 客服 态度 热情 价格 实惠,两件套 套装,M 建议 100 110 斤,2022-10-24,两件套套装
2,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****g,False,5,质量 真的 不错 满意 包装 用心 客服 态度 特别 值得 推荐,两件套 套装,M 建议 100 110 斤,2022-10-23,两件套套装
3,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****C,False,5,衣服 收到 套装 懒人 福音 不用 搭配 质量 穿 特别 显瘦 气质 通勤 日常 穿 好看 ...,两件套 套装,M 建议 100 110 斤,2022-10-23,两件套套装
4,服饰内衣,女装,连衣裙,皮尔卡丹（pierre cardin）,158.00,97.00%,****E,False,5,面料 舒适 尺码 大小 标准 做工 精细 客服 态度 特别,两件套 套装,M 建议 100 110 斤,2022-10-22,两件套套装


In [26]:
df2 = df.copy()

切分size属性

In [27]:
# df['style'].str.split(expand=True)
# df['size'].str.split(expand=True)
import re
for index, row in df.iterrows():
    if '码' in row['size']:
        df2.loc[index,'size'] = re.sub('(?<=码).*',"", row['size']).strip().replace(" ","")
    else:
        df2.loc[index,'size'] = re.sub(r"[^a-zA-Z ]+", '', row['size']).strip().replace(" ","")
        
df2['size'].unique()

array(['M', 'F', 'L', '均码', 'AL', 'XL', 'XXL', 'S', 'XXXL', 'AM', '',
       'XL码', 'AXL', 'AXS', 'AS', 'A', 'MA', 'LA', 'XXS', 'XLA', 'XXLA',
       'AXXL', '36码', '2XL码', '33码', 'B', '请拍正确尺码', 'XS', '大码', '加大码',
       'S码', 'AB', 'BAB', 'ABCM', 'Free', 'MABCA', 'C', 'BM', '75B34B配M码',
       'ABC', 'XLABC', 'BCDcm', 'CDcm', 'BCD', '7534AB通杯偏小一码', 'D', 'DD',
       'MCAB', 'CM', 'MABC', 'AA', 'ABAB', 'LABC', 'BC', '75B内裤M码',
       'uminiB', 'miniB', 'BB', 'CC', 'XXLABC', 'M均码', 'MCDBCD', 'ACM',
       'LABCD', 'MBCAB', 'cm', '150码', '140码', '160码', '130码', 'Mcm',
       '28码', '37码', 'mm', 'cmcm', '24码', '5Y37.5码', '34码', '32码',
       '9.5k27码', '27码', '内长12.5cm18码', '38码', '29码', '39码', '31码', '13码',
       '4.5uk37码', '30码', '20码', '17码', '38.5码', '05Y37.5码', '10.5k28.5码',
       '内长15.5cm25码', '33.5码', '2.5uk35码', '13C31码', '35码', '16码', '15码',
       '11C28码'], dtype=object)

In [28]:
df2.to_csv("评论详情表.csv",encoding="utf_8_sig")

#### 输出评论详情表

In [29]:
df.to_csv("评论详情表.csv",encoding="utf_8_sig")

### 提取额外信息

#### 获得价格分组和销售数

In [40]:
df['price'] = df['price'].str.replace("￥","")

In [41]:
df["price"] = df["price"].apply(pd.to_numeric)
df_group_sales = df.groupby(["first_category","second_category","third_category","brand"])['price'].value_counts().rename('counts').reset_index()

#### 统计销售额

In [42]:
df_group_sales["sales"]=df_group_sales[["price","counts"]].apply(lambda x:x["price"] * x["counts"],axis=1)

In [43]:
df_group_sales['sales']

0         30.3
1       1040.4
2        645.0
3        268.8
4        257.4
        ...   
856     1432.8
857     1245.0
858      995.0
859     4450.0
860    19450.0
Name: sales, Length: 861, dtype: float64

#### 统计同类产品的销售天数

In [44]:
df_group_days = df.groupby(["first_category","second_category","third_category","brand","price"])

In [45]:
df_group_days = df_group_days['comment_time'].nunique().rename('days').reset_index()
pd.DataFrame(df_group_days)
df_group_days

,first_category,second_category,third_category,brand,price,days
0,图书,时尚/美妆,服饰搭配,凤凰含章,30.3,1
1,图书,时尚/美妆,服饰搭配,联合读创,61.2,17
2,宠物生活,猫狗出行,宠物配饰,派乐特,12.9,50
3,家居日用,婚庆节庆,其它婚庆节庆用品,拾点渔,12.8,20
4,家居日用,生活日用,缝纫/针织材料,阿麦思,9.9,26
...,...,...,...,...,...,...
856,运动户外,运动服饰,运动配饰,迪卡侬（DECATHLON）,19.9,0
857,运动户外,运动服饰,运动配饰,迪卡侬（DECATHLON）,24.9,0
858,运动户外,运动服饰,运动配饰,里海狼（CASPIAN WOLF）,19.9,50
859,运动户外,运动服饰,运动配饰,阿迪达斯 （adidas）,89.0,1


#### 合并销售额和天数

In [46]:
df = pd.merge(df_group_sales,df_group_days, how = "inner",on = ["first_category","second_category","third_category","brand","price"])
df

,first_category,second_category,third_category,brand,price,counts,sales,days
0,图书,时尚/美妆,服饰搭配,凤凰含章,30.3,1,30.3,1
1,图书,时尚/美妆,服饰搭配,联合读创,61.2,17,1040.4,17
2,宠物生活,猫狗出行,宠物配饰,派乐特,12.9,50,645.0,50
3,家居日用,婚庆节庆,其它婚庆节庆用品,拾点渔,12.8,21,268.8,20
4,家居日用,生活日用,缝纫/针织材料,阿麦思,9.9,26,257.4,26
...,...,...,...,...,...,...,...,...
856,运动户外,运动服饰,运动配饰,迪卡侬（DECATHLON）,19.9,72,1432.8,0
857,运动户外,运动服饰,运动配饰,迪卡侬（DECATHLON）,24.9,50,1245.0,0
858,运动户外,运动服饰,运动配饰,里海狼（CASPIAN WOLF）,19.9,50,995.0,50
859,运动户外,运动服饰,运动配饰,阿迪达斯 （adidas）,89.0,50,4450.0,1


In [47]:
df.to_csv("销售详情表.csv",encoding="utf_8_sig")